In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor


In [ ]:
years = list(range(2024, 2019, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
visited_urls = set()

for year in years:
    print(f"Fetching data for season {year}...")
    try:
        # Fetch the standings page
        data = requests.get(standings_url)
        if standings_url in visited_urls:
            print("Repeated URL detected. Stopping the loop.")
            break
        visited_urls.add(standings_url)

        soup = BeautifulSoup(data.text, 'html.parser')
        standings_table = soup.select('table.stats_table')[0]

        # Extract team URLs
        links = [l.get("href") for l in standings_table.find_all('a')]
        links = [l for l in links if '/squads/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        # Get the previous season URL
        prev_links = soup.select("a.prev")
        if not prev_links:
            print("No previous season link found. Stopping the loop.")
            break
        previous_season = prev_links[0].get("href")
        standings_url = f"https://fbref.com{previous_season}"

        # Loop through each team and get data
        for team_url in team_urls:
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            print(f"Fetching data for {team_name}...")
            try:
                data = requests.get(team_url)
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
                soup = BeautifulSoup(data.text, 'html.parser')

                # Get shooting stats
                links = [l.get("href") for l in soup.find_all('a')]
                links = [l for l in links if l and 'all_comps/shooting/' in l]
                if not links:
                    print(f"No shooting data found for {team_name}")
                    continue

                shooting = pd.read_html(f"https://fbref.com{links[0]}", match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()

                # Merge matches and shooting data
                team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how='left')
                team_data = team_data[team_data["Comp"] == "Premier League"]
                team_data["Season"] = year
                team_data["Team"] = team_name
                all_matches.append(team_data)

            except Exception as e:
                print(f"Error fetching data for {team_name}: {e}")

            time.sleep(10)

    except Exception as e:
        print(f"Error fetching standings for season {year}: {e}")
        break

# Combine all data
if all_matches:
    final_df = pd.concat(all_matches, ignore_index=True)
    print("Data fetching completed successfully!")
else:
    print("No data fetched.")

In [ ]:
print(team_data)

In [ ]:
ttable = pd.concat(all_matches)

In [ ]:
ttable['GF'] = ttable['GF'].fillna(0).astype(int)
ttable['GA'] = ttable['GA'].fillna(0).astype(int)
ttable=ttable.reset_index()
ttable.drop('index',axis=1,inplace=True)

In [ ]:
ttable.to_csv('ttable.csv')

In [ ]:
years = list(range(2024, 2019, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')

    if not standings_table:
        print(f"No league table found for {standings_url}")
        continue
    
    standings_table = standings_table[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")
    if previous_season:
        standings_url = f"https://fbref.com{previous_season[0].get('href')}"
    else:
        print(f"No previous season link found for {standings_url}")
        break
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        
        
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        
        if not links:
            print(f"No passing data link found for {team_name} in {year}")
            continue
        
        data = requests.get(f"https://fbref.com{links[0]}")
        passing = pd.read_html(StringIO(data.text), match="Passing")[0]
        passing.columns = passing.columns.droplevel()
        
       
        try:
            team_data1 = matches.merge(
                passing[["Date", "KP", "1/3", "PPA", "CrsPA", "PrgP"]], 
                on="Date", how="left"
            )
        except KeyError as e:
            print(f"KeyError for merging data of {team_name} in {year}: {e}")
            continue
        
        team_data1 = team_data1[team_data1["Comp"] == "Premier League"]
        team_data1["Season"] = year
        team_data1["Team"] = team_name
        all_matches.append(team_data1)
        time.sleep(10)


if all_matches:
    ttable = pd.concat(all_matches, ignore_index=True)
    print(f"Collected data for {len(ttable)} matches.")
else:
    print("No matches collected.")


In [ ]:
assists = pd.concat(all_matches)
assists=assists.reset_index()
assists.drop('index',axis=1,inplace=True)

In [ ]:
assists.to_csv('assists.csv')

In [ ]:
years = list(range(2024, 2019, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')

    if not standings_table:
        print(f"No league table found for {standings_url}")
        continue
    
    standings_table = standings_table[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")
    if previous_season:
        standings_url = f"https://fbref.com{previous_season[0].get('href')}"
    else:
        print(f"No previous season link found for {standings_url}")
        break
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        
        
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/possession/' in l]
        
        if not links:
            print(f"No possession data link found for {team_name} in {year}")
            continue
        
        data = requests.get(f"https://fbref.com{links[0]}")
        possession = pd.read_html(StringIO(data.text), match="Possession")[0]
        possession.columns = possession.columns.droplevel()

        
        required_columns = ["Att 3rd", "Att Pen", "PrgC"]
        available_columns = [col for col in required_columns if col in possession.columns]

        if not available_columns:
            print(f"Required possession columns not found for {team_name} in {year}")
            continue

        try:
            
            team_data2 = matches.merge(possession[["Date"] + available_columns], on="Date")
        except ValueError as e:
            print(f"Merge error for {team_name} in {year}: {e}")
            continue

        team_data2 = team_data2[team_data2["Comp"] == "Premier League"]
        team_data2["Season"] = year
        team_data2["Team"] = team_name
        all_matches.append(team_data2)
        time.sleep(10)


if all_matches:
    ttable = pd.concat(all_matches, ignore_index=True)
    print(f"Collected data for {len(ttable)} matches.")
else:
    print("No matches collected.")


In [ ]:
posses = pd.concat(all_matches)
posses=posses.reset_index()
posses.drop('index',axis=1,inplace=True)

In [ ]:

posses.to_csv('possession.csv', index=False)


In [ ]:
years = list(range(2024, 2019, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')

    if not standings_table:
        print(f"No league table found for {standings_url}")
        continue
    
    standings_table = standings_table[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")
    if previous_season:
        standings_url = f"https://fbref.com{previous_season[0].get('href')}"
    else:
        print(f"No previous season link found for {standings_url}")
        break
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        
        
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/gca/' in l]
        
        if not links:
            print(f"No goal and shot creation data link found for {team_name} in {year}")
            continue
        
        data = requests.get(f"https://fbref.com{links[0]}")
        gfc = pd.read_html(StringIO(data.text), match="Goal and Shot Creation")[0]
        gfc.columns = gfc.columns.droplevel()

       
        if "Date" not in gfc.columns or "SCA" not in gfc.columns:
            print(f"Required columns not found for {team_name} in {year}")
            continue

        try:
            
            team_data3 = matches.merge(gfc[["Date", "SCA"]], on="Date")
        except ValueError as e:
            print(f"Merge error for {team_name} in {year}: {e}")
            continue

        team_data3 = team_data3[team_data3["Comp"] == "Premier League"]    
        team_data3["Season"] = year
        team_data3["Team"] = team_name
        all_matches.append(team_data3)
        time.sleep(10)


if all_matches:
    ttable = pd.concat(all_matches, ignore_index=True)
    print(f"Collected data for {len(ttable)} matches.")
else:
    print("No matches collected.")


In [ ]:
creation = pd.concat(all_matches)
creation=creation.reset_index()
creation.drop('index',axis=1,inplace=True)

In [ ]:
creation.to_csv('creation.csv')

In [ ]:
years = list(range(2024, 2019, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    
    
    standings_table = soup.select('table.stats_table')
    if not standings_table:
        print(f"No table found for {standings_url}")
        continue
    
    standings_table = standings_table[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    
    previous_season = soup.select("a.prev")
    if not previous_season:
        print(f"No previous season link found for {standings_url}")
        break
    
    standings_url = f"https://fbref.com{previous_season[0].get('href')}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        
        
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing_types/' in l]
        
        if not links:
            print(f"No passing type link found for {team_name} in {year}")
            continue
        
        data = requests.get(f"https://fbref.com{links[0]}")
        passing_type = pd.read_html(StringIO(data.text), match="Pass Types")[0]
        passing_type.columns = passing_type.columns.droplevel()
        
        
        if "Date" not in passing_type.columns or "TB" not in passing_type.columns:
            print(f"Required columns not found for {team_name} in {year}")
            continue
        
        try:
            
            team_data5 = matches.merge(passing_type[["Date", "TB"]], on="Date")
        except ValueError as e:
            print(f"Merge error for {team_name} in {year}: {e}")
            continue
        
        team_data5 = team_data5[team_data5["Comp"] == "Premier League"]
        team_data5["Season"] = year
        team_data5["Team"] = team_name
        all_matches.append(team_data5)
        time.sleep(10)


if all_matches:
    full_table = pd.concat(all_matches, ignore_index=True)
    print(f"Collected data for {len(full_table)} matches.")
else:
    print("No matches collected.")


In [ ]:
back1=pd.concat(all_matches)
back1=back1.reset_index()
back1.drop('index',axis=1,inplace=True)

In [ ]:
back1.to_csv('back1.csv', index=False)

In [ ]:
ttable = pd.read_csv('ttable.csv')
assists = pd.read_csv('assists.csv')
posses = pd.read_csv('possession.csv')
creation = pd.read_csv('creation.csv')
back1 = pd.read_csv('back1.csv')

In [ ]:
def season(x):
    mainseas = ttable[ttable['Season'] == x]
    ass = assists[assists['Season'] == x]
    poss = posses[posses['Season'] == x]
    cre = creation[creation['Season'] == x]
    back = back1[back1['Season'] == x]

    feat = ["Sh", "SoT", "Dist", "FK", "PK", "PKatt", 'GF', 'GA', 'xG', 'xGA', 'Poss',
            "KP", "1/3", "PPA", "CrsPA", "Prog", "Att 3rd", "Att Pen", "Prog", 'SCA', 'TB']
    df = pd.DataFrame()
    df['Team'] = mainseas['Team'].unique()
    
    # Initialize columns with zeros
    for i in feat:
        df[i] = 0.0

    main = ["Sh", "SoT", "Dist", "FK", "PK", "PKatt", 'GF', 'GA', 'xG', 'xGA', 'Poss']
    asss = ["KP", "1/3", "PPA", "CrsPA", "PrgP"]
    posss = ["Att 3rd", "Att Pen", "PrgC"]
    gfc = ['SCA']
    tb = ['TB']
    feat1 = [main, asss, poss, gfc, tb]
    tables = [mainseas, ass, poss, cre, back]


    # Assign main features
    for j in main:
        for i in range(len(df)):
            df.loc[i, j] = mainseas[mainseas['Team'] == df.loc[i, 'Team']][j].mean()

    # Assign assist features
    for j in asss:
        for i in range(len(df)):
            df.loc[i, j] = ass[ass['Team'] == df.loc[i, 'Team']][j].mean()

    # Assign possession features
    for j in posss:
        for i in range(len(df)):
            df.loc[i, j] = poss[poss['Team'] == df.loc[i, 'Team']][j].mean()

    # Special feature assignments
    for i in range(len(df)):
        df.loc[i, 'SCA'] = cre[cre['Team'] == df.loc[i, 'Team']]['SCA'].mean()
    for i in range(len(df)):
        df.loc[i, 'TB'] = back[back['Team'] == df.loc[i, 'Team']]['TB'].mean()

    # Add Season, Points, and Points per match columns
    df['Season'] = x
    df['Points'] = 0
    df['Points per match'] = 0.0
    
    for i in range(len(df)):
        win_count = mainseas[mainseas['Team'] == df.loc[i, 'Team']]['Result'].value_counts().get('W', 0)
        draw_count = mainseas[mainseas['Team'] == df.loc[i, 'Team']]['Result'].value_counts().get('D', 0)
        df.loc[i, 'Points'] = win_count * 3 + draw_count
    
    for i in range(len(df)):
        df.loc[i, 'Points per match'] = df.loc[i, 'Points'] / len(mainseas[mainseas['Team'] == df.loc[i, 'Team']])
    
    return df

# Creating DF for different seasons
seas19 = season(2019)
seas20 = season(2020)
seas21 = season(2021)
seas22 = season(2022)
seas23 = season(2023)
seas24 = season(2024)

# Concatenating all the seasons
total = pd.concat([seas19, seas20, seas21, seas22, seas23, seas24])


In [ ]:
total.to_csv('Final_Data.csv')

In [ ]:
print(total.columns)

In [ ]:
print(total.head())

In [ ]:
def fetch_url(url, sleep_time=10):
    """Fetch the URL with retries and delay."""
    for _ in range(3):  
        response = requests.get(url)
        if response.status_code == 200:
            return response
        print(f"Failed request with status {response.status_code}. Retrying...")
        time.sleep(sleep_time)
    return None

def scrape_premier_league_data():
    standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
    years = list(range(2024, 2018, -1))
    all_matches = []

    for year in years:
        print(f"Fetching data for season {year}...")
        response = fetch_url(standings_url)
        if not response:
            print(f"Skipping {year} due to repeated request failures.")
            continue

        time.sleep(10)
        soup = BeautifulSoup(response.text, "html.parser")
        standings_table = soup.select('table.stats_table')
        if not standings_table:
            print(f"No standings table found for {year}.")
            continue

        links = [l.get("href") for l in standings_table[0].find_all('a') if l.get("href") and '/squads/' in l.get("href")]
        team_urls = [f"https://fbref.com{l}" for l in links]

        
        previous_season = soup.select("a.prev")
        if previous_season:
            standings_url = f"https://fbref.com{previous_season[0].get('href')}"
        else:
            print(f"No previous season link found for {year}.")
            break

        for team_url in team_urls:
            try:
                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
                team_response = fetch_url(team_url)
                if not team_response:
                    print(f"Skipping {team_name} due to request failure.")
                    continue
                time.sleep(10)
                
                matches = pd.read_html(team_response.text, match="Scores & Fixtures")[0]
                
                soup_team = BeautifulSoup(team_response.text, "html.parser")
                
                
                shooting_links = [l.get("href") for l in soup_team.find_all('a') if l and 'all_comps/shooting/' in l]
                if shooting_links:
                    shooting_response = fetch_url(f"https://fbref.com{shooting_links[0]}")
                    if shooting_response:
                        shooting = pd.read_html(shooting_response.text, match="Shooting")[0]
                        shooting.columns = shooting.columns.droplevel()
                        matches = matches.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on="Date", how="left")
                
                
                passing_links = [l.get("href") for l in soup_team.find_all('a') if l and 'all_comps/passing/' in l]
                if passing_links:
                    passing_response = fetch_url(f"https://fbref.com{passing_links[0]}")
                    if passing_response:
                        passing = pd.read_html(passing_response.text, match="Passing")[0]
                        passing.columns = passing.columns.droplevel()
                        matches = matches.merge(passing[['Date', 'KP', '1/3', 'PPA', 'CrsPA', 'Prgp']], on="Date", how="left")
                
                
                matches["Season"] = year
                matches["Team"] = team_name
                all_matches.append(matches)

            except Exception as e:
                print(f"Error processing team {team_name} in {year}: {e}")
                continue

    
    final_df = pd.concat(all_matches, ignore_index=True)
    final_df.columns = [c.lower() for c in final_df.columns]
    return final_df


In [ ]:
final_df.to_csv('fix.csv')

In [ ]:
ttotal = pd.read_csv(r'FFinal_Data.csv')

In [ ]:
filtered_total = ttotal.drop(columns=['Prog', 'sno'])

In [ ]:
filtered_total_numeric = filtered_total.select_dtypes(include=['number'])

# Adjust size dynamically based on the number of numeric columns
num_cols = filtered_total_numeric.shape[1]
figsize = (24, 24 * 0.6)

# Create the heatmap
plt.figure(figsize=figsize)
sns.heatmap(filtered_total_numeric.corr(), annot=True, cmap='coolwarm')

plt.show()


In [ ]:
plt.savefig(r'heatmap.png', bbox_inches='tight')

In [ ]:
from sklearn.model_selection import train_test_split
X=ttotal.drop(['Points','Season','Team','PpG'],axis=1)
y=ttotal['Points']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
seas24_test=seas24

In [ ]:
print(seas24.head())

In [ ]:
knn = KNeighborsRegressor(n_neighbors=12)
dt  = DecisionTreeRegressor(max_depth = 5)
rf  = RandomForestRegressor(n_estimators=100, max_features= 7)
ada = AdaBoostRegressor( n_estimators=150, learning_rate =.08)
gbr = GradientBoostingRegressor(max_depth=7, n_estimators=500, learning_rate =.05)
xgb = XGBRegressor(max_depth = 7, n_estimators=500, learning_rate =.05)
cb  = CatBoostRegressor(learning_rate =.07, max_depth =7, verbose=0)

regressors = [('K Nearest Neighbours', knn),('Decision Tree', dt), ('Random Forest', rf), 
              ('AdaBoost', ada),('Gradient Boosting Regressor', gbr),('XGBRegressor', xgb),('CatBoostRegressor', cb)]

In [ ]:
from sklearn.metrics import r2_score

for regressor_name, regressor in regressors:
    regressor.fit(X_train, y_train)  
    y_pred = regressor.predict(X_test)
    accuracy = round(r2_score(y_test, y_pred), 3) * 100
    
    print('{:s} : {:.0f} %'.format(regressor_name, accuracy))
    
    
    random_color = [random.random() for _ in range(3)]
    plt.rcParams["figure.figsize"] = (20, 8)
    plt.bar(regressor_name, accuracy, color=random_color)
    
    
    plt.text(regressor_name, accuracy + 1, f'{accuracy:.0f}%', ha='center', fontsize=12)

plt.title('Model Accuracy Comparison')
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.show()


In [ ]:
plt.savefig(r'reg_accu.png', bbox_inches='tight')

In [ ]:
seas24_copy=seas24_test

In [ ]:
print(seas24_copy.head())

In [ ]:
seas24_copy.drop(['Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt', 'GF', 'GA', 'xG',
       'xGA', 'Poss', 'KP', '1/3', 'PPA', 'CrsPA' , 'Att 3rd',
       'Att Pen', 'SCA', 'TB', 'Prog'],axis=1,inplace=True)

In [ ]:
print(seas24_copy.head())

In [ ]:
fix=pd.read_csv(r'fix.csv')

In [ ]:
seas24_copy['Predicted Points XGB']=0.0
for i in range(len(seas24_copy['Team'].unique())):
    seas24_copy['Predicted Points XGB'][i]=seas24_copy['Points per match'][i]*(38-len(fix[(fix['Team']==seas24_copy['Team'][i])&(fix['Season']==2024)]))+seas24_copy['Points'][i]

In [ ]:
seas24_copy['Predicted Points AdaBoost']=0.0
for i in range(len(seas24_copy['Team'].unique())):
    seas24_copy['Predicted Points AdaBoost'][i]=seas24_copy['Points per match'][i]*(38-len(fix[(fix['Team']==seas24_copy['Team'][i])&(fix['Season']==2024)]))+seas24_copy['Points'][i]

In [ ]:
seas24_copy['Predicted Points Random Forest']=0.0
for i in range(len(seas24_copy['Team'].unique())):
    seas24_copy['Predicted Points Random Forest'][i]=seas24_copy['Points per match'][i]*(38-len(fix[(fix['Team']==seas24_copy['Team'][i])&(fix['Season']==2024)]))+seas24_copy['Points'][i]

In [ ]:
seas24_1=seas24_copy

In [ ]:
seas24_1[['Predicted Points XGB','Predicted Points AdaBoost','Predicted Points Random Forest']]=seas24_1[['Predicted Points XGB','Predicted Points AdaBoost','Predicted Points Random Forest']].astype(int)

In [ ]:
seas24_1.sort_values('Predicted Points XGB',ascending=False)[['Team','Season','Predicted Points XGB']]

In [ ]:
seas24_1.sort_values('Predicted Points AdaBoost',ascending=False)[['Team','Season','Predicted Points AdaBoost']]

In [ ]:
seas24_1.sort_values('Predicted Points Random Forest',ascending=False)[['Team','Season','Predicted Points Random Forest']]